In [0]:
# import section
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('orders').getOrCreate()


In [0]:
%run /Workspace/apache-spark/databricks-project-fmcg-sports/utils/utilities

In [0]:
# setup the catlog, datasource and s3 paths
dbutils.widgets.text('catlog','fmcg','catlog')
dbutils.widgets.text('datasource','orders','datasource')

catlog      = dbutils.widgets.get('catlog')
datasource  = dbutils.widgets.get('datasource')

base_path   = f'{s3_bucket}/{datasource}'
landing_path = f'{base_path}/landing'
processed_path = f'{base_path}/processed'

#print(catlog,datasource,base_path, landing_path, processed_path)

In [0]:
# read orders full load
orders_silver = (
    spark.read.format('csv')
    .option('header','true')
    .option('inferSchema','true')
    .load(f'{landing_path}/*.csv')
    .withColumns({
        'load_timestamp': f.current_timestamp(),
        'file_name': f.col('_metadata.file_name'),
        'file_size': f.col('_metadata.file_size')
    })
)
orders_silver.count()

In [0]:
orders_silver.limit(10).display()